In [1]:

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from unidecode import unidecode
## ler arquivo json em pandas
df = pd.read_json('/root/projects/personal/comitiva_esperanca/label-studio/data/export/project-1-at-2023-05-07-03-57-20fa509e.json')

df = df.drop(['id'], axis=1)
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
df = df.drop(['id'], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df['choices'] != 'Desclassificar']
df.count()

file_upload                 294
drafts                      294
predictions                 294
meta                        294
created_at                  294
updated_at                  294
inner_id                    294
total_annotations           294
cancelled_annotations       294
total_predictions           294
comment_count               294
unresolved_comment_count    294
last_comment_updated_at       0
project                     294
updated_by                  294
comment_authors             294
completed_by                294
was_cancelled               294
ground_truth                294
created_at                  294
updated_at                  294
lead_time                   294
prediction                  294
result_count                294
unique_id                   294
last_action                   0
task                        294
project                     294
updated_by                  294
parent_prediction             0
parent_annotation             0
last_cre

In [2]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)

df_treino = df[:210]
df_validacao = df[210:]

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoModel, AutoTokenizer

# Criar modelo de classificação de sentimento
vocab_size = 1000
embedding_dim = 100
max_length = 125
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Pré-processar os dados
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df['noticia'])
sequences = tokenizer.texts_to_sequences(df['noticia'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
labels = df['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 0.5})

# Treinar o modelo
history = model.fit(padded_sequences, labels, epochs=100, validation_split=0.3)

# Avaliar o modelo
test_sequences = tokenizer.texts_to_sequences(df_validacao['noticia'])
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')
test_labels = df_validacao['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 0.5})
test_loss, test_accuracy = model.evaluate(padded_test_sequences, test_labels)
print('Test accuracy:', test_accuracy)

# Salvar o modelo
model.save('modelo.h5')

2023-05-07 01:05:54.946062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 01:05:55.407216: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 01:05:55.408928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 01:05:57.057716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/root/projects/personal/comitiva_esperanca/.python_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023

Epoch 1/100
7/7 [==============================] - 1s 46ms/step - loss: 0.6932 - accuracy: 0.4000 - val_loss: 0.6926 - val_accuracy: 0.4270
Epoch 2/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6907 - accuracy: 0.4780 - val_loss: 0.6922 - val_accuracy: 0.4270
Epoch 3/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6884 - accuracy: 0.4780 - val_loss: 0.6916 - val_accuracy: 0.4270
Epoch 4/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6855 - accuracy: 0.4780 - val_loss: 0.6912 - val_accuracy: 0.4270
Epoch 5/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6817 - accuracy: 0.4780 - val_loss: 0.6902 - val_accuracy: 0.4270
Epoch 6/100
7/7 [==============================] - 0s 9ms/step - loss: 0.6776 - accuracy: 0.4780 - val_loss: 0.6890 - val_accuracy: 0.4270
Epoch 7/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6724 - accuracy: 0.4780 - val_loss: 0.6867 - val_accuracy: 0.4270
Epoch 8/100
7/7 [=========

In [3]:
## criar uma tabela de predições
df_predicoes = df_validacao.copy()
df_predicoes['predicao'] = model.predict(padded_test_sequences)
df_predicoes['predicao'] = df_predicoes['predicao'].apply(lambda x: 'Positiva' if x > 0.5 else 'Negativa' if x < 0.5 else 'Neutra')
df_predicoes

##  quantas noticias foram classificadas como positivas, negativas e neutras
df_predicoes['predicao'].value_counts()

## quantas noticias foram classificadas erradas
df_predicoes

3/3 [==============================] - 0s 2ms/step


,id,data,noticia,titulo,choices,unique_id,predicao
246,9jPu-wLJ6L,27/05/2019,maiores demandas interna externa farelo oleo s...,SOJA/CEPEA: Demanda aquecida por farelo e óleo...,Positiva,701c2b37-164f-4dc7-980c-e81234625d92,Positiva
247,ASTg8bJjN5,,boa parte safra soja brasileira ja comercializ...,"SOJA/CEPEA: Com menor ritmo de embarques, prêm...",Negativa,85ba023a-e6c6-4768-967a-58d52de40d92,Negativa
249,NVXB7cSsqU,17/10/2022,precos soja voltaram subir mercado interno sem...,SOJA/CEPEA: Valores reagem no Brasil,Positiva,1e020d17-fb5c-4052-8774-67b431cda161,Negativa
250,oWw-sNX79i,,valorizacao dolar frente real elevado forca pr...,SOJA/CEPEA: Dólar sobe e eleva média mensal do...,Positiva,7fa6a610-b6e9-4abf-95c5-22ba119630de,Negativa
251,TndmtqGgQA,2/10/2017,precos soja derivados subiram brasil setembro ...,SOJA/CEPEA: Demanda externa elevada e recuo pr...,Positiva,6ef899b3-cd6b-4e8d-8675-3a8e9c2db44e,Positiva
...,...,...,...,...,...,...,...
433,67nRmTzCS7,,menor demanda externa cautela compradores dome...,SOJA/CEPEA: Com menor procura e frete rodoviár...,Negativa,4a09d340-1854-4968-a92c-fb5de9599843,Positiva
446,qUVxbRQnyQ,08/05/2017,fim colheita safra nacional recorde precos soj...,SOJA/CEPEA: Cotações avançam pela 4º semana co...,Positiva,b7430298-2d42-4188-9fbc-48f9fc2c544a,Positiva
449,S5JS_zabnl,26/10/2015,valorizacao dolar ultima semana produtores vol...,SOJA/CEPEA: Preços sobem; clima melhora e favo...,Positiva,70468e40-8c29-4681-909a-29c200fa6b52,Positiva
460,buwb5cnqIJ,15/02/2017,precos soja derivados cairam brasil ultimos di...,SOJA/CEPEA: Queda nos preços é limitada por dó...,Neutra,6659f9f6-b7a5-4b2c-a322-b3fca135c13a,Negativa


In [4]:
## prever uma noticia
df_validacao = df_validacao.reset_index(drop=True)
noticia = df_validacao['noticia'][0]
noticia = preprocess_text(noticia)
noticia = [noticia]
noticia = tokenizer.texts_to_sequences(noticia)
noticia = pad_sequences(noticia, maxlen=max_length, padding='post', truncating='post')
val = model.predict(noticia)[0][0]


if val > 0.5:
    print('Positiva')
elif val < 0.5:
    print('Negativa')
else:
    print('Neutra')



1/1 [==============================] - 0s 17ms/step
Positiva


In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Carregar o modelo pré-treinado BERTimbau
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

model
# device = torch.device('cpu')

# # Definir otimizador e função de perda
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

# train_sentences = df_treino['noticia'].tolist()
# train_labels = df_treino['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 2}).tolist()
# test_sentences = df_validacao['noticia'].tolist()
# test_labels = df_validacao['choices'].map({'Positiva': 1, 'Negativa': 0, 'Neutra': 2}).tolist()

# # Tokenizar as sentenças
# train_encodings = tokenizer(train_sentences, truncation=True, padding=True, return_tensors='pt')
# test_encodings = tokenizer(test_sentences, truncation=True, padding=True, return_tensors='pt')

# # Criar um dataset do pytorch
# class MyDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_dataset = MyDataset(train_encodings, train_labels)
# test_dataset = MyDataset(test_encodings, test_labels)

# # Treinar o modelo
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
# for epoch in range(2):
#     for i, batch in enumerate(train_loader):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optimizer.step()

# # Avaliar o modelo nos dados de teste
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
# model.eval()
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for batch in test_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask)
#         predictions = torch.argmax(outputs[0], axis=1)
#         total += labels.size(0)
#         correct += (predictions == labels).sum().item()
#     accuracy = correct / total
#     print('Test accuracy:', accuracy)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,